In [103]:
from matplotlib import pyplot as plt
from skimage.feature import hog
from skimage import exposure
from skimage.transform import resize

from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

import os
import numpy as np
import nibabel as nib

import pandas as pd


In [123]:
def process_img(filepath, resize_l, resize_w, resize_d):
    scan = nib.load(filepath)
    scan = scan.get_fdata()
    
    scan = resize(scan, output_shape=(resize_l, resize_w, resize_d))
    scan = hog(scan, orientations=16, pixels_per_cell=(20,20), cells_per_block=(1,1), channel_axis=-1)
    return scan


In [124]:
resize_l = 250
resize_w = 250
resize_d = 64

data_folder = f'{os.getcwd()}/labeled_data/data/'

length = 0
for label in os.listdir(data_folder):
    length += len(os.listdir(f'{data_folder}/{label}/'))


# X = np.empty(shape=(length, resize_l, resize_w, 64,), dtype=np.float16)
X = []
Y = []

print('Loading Images')
for label in os.listdir(data_folder):
    img_directory = f'{data_folder}/{label}/'
    for img in os.listdir(img_directory): 
        
        img_path = '{}{}'.format(img_directory, img)
        img = process_img(img_path, resize_l, resize_w, resize_d)
        X += [img]
        Y += [label]
        

Loading Images


In [125]:
train_x, test_x, train_y, test_y = train_test_split(X, Y, test_size=0.3, random_state=777)


In [126]:
classifier = SVC()

classifier.fit(train_x, train_y)
y_pred = classifier.predict(test_x)

acc = np.mean(y_pred == test_y)
acc

0.7317880794701986

In [150]:
data_folder = f'{os.getcwd()}/ATR_data/data/'

ground_truth = pd.read_excel('testing_labels.xlsx', names=['filename'])
gt_x = []

for i in ground_truth.index:
    filename = ground_truth['filename'].iloc[i]

    img_path = '{}{}.nii.gz'.format(data_folder,filename[1:-1])
    gt_x += [process_img(img_path, resize_l, resize_w, resize_d)]

gt_y = classifier.predict(gt_x)



In [151]:
gt_y

array(['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
       '0', '2', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0

In [139]:
ground_truth

,filename
0,'I135_2'
1,'I135_3'
2,'I135_4'
3,'I135_5'
4,'I135_6'
...,...
461,'I193_18'
462,'I193_19'
463,'I193_20'
464,'I193_21'
